##Tải về tập dữ liệu flickr8k đã clean

In [ ]:
!git clone https://github.com/DoanNgocToan/clean_data_flickr8k

Cloning into 'clean_data_flickr8k'...
remote: Enumerating objects: 8128, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 8128 (delta 2), reused 0 (delta 0), pack-reused 8125 (from 3)
Receiving objects: 100% (8128/8128), 1.07 GiB | 46.90 MiB/s, done.
Resolving deltas: 100% (15/15), done.
Updating files: 100% (8096/8096), done.


In [ ]:
!ls /content/clean_data_flickr8k/

captions.txt  features.rar  Images  README.md  vocabulary.txt


##Thiết lập đường dẫn

In [ ]:
import os
from collections import defaultdict
from IPython.display import display, Image

dataset_path = '/content/clean_data_flickr8k'
caption_path = os.path.join(dataset_path, 'captions.txt')
image_dirs = os.path.join(dataset_path, 'Images')

# Create a directory to save image paths and captions
image_paths = set()

if os.path.exists(caption_path):
  with open(caption_path, 'r') as f:
    next(f) # Bỏ qua tiêu đề 'image,caption'
    for line in f:
      # Split only at the first comma to correctly separate image and caption
      dir_cap = line.strip().split(',', 1)
      image_path = os.path.join(image_dirs, dir_cap[0])
      image_paths.add(image_path)

##Tiền xử lý



In [ ]:
import os
import pickle
import numpy as np
import tqdm.notebook as tqdm
!pip install tensorflow
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Model
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, add

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.7/620.7 MB 737.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 70.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 88.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 225.0/225.0 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 6.7 MB/s eta 0:00:00


/usr/local/lib/python3.12/dist-packages/jax/_src/cloud_tpu_init.py:86: UserWarning: Transparent hugepages are not enabled. TPU runtime startup and shutdown time should be significantly improved on TPU v5e and newer. If not already set, you may need to enable transparent hugepages in your VM image (sudo sh -c "echo always > /sys/kernel/mm/transparent_hugepage/enabled")
  warnings.warn(


In [ ]:
model = VGG16()
# restructure the model
model = Model(inputs=model.inputs, outputs=model.layers[-2].output)
# summarize
print(model.summary())

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc1 (Dense)                     │ (None, 4096)           │   102,764,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fc2 (Dense)                     │ (None, 4096)           │    16,781,312 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 134,260,544 (512.16 MB)

 Trainable params: 134,260,544 (512.16 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
features = {}
directory = image_dirs;

for img_name in tqdm.tqdm(os.listdir(directory)):
  img_path = directory + '/' + img_name
  image = load_img(img_path, target_size=(224, 224))
  # convert image pixel to numpy array
  image = img_to_array(image)
  # reshape data for model
  image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
  # preprocess image for VGG model
  image = preprocess_input(image)
  # extract features
  feature = model.predict(image, verbose=0)
  # get image ID
  image_id = img_name.split('.')[0]
  # store feature
  features[image_id] = feature

  0%|          | 0/8091 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
!apt-get install unrar-free

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unrar-free is already the newest version (1:0.0.2-0.1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [ ]:
import os

# Navigate to the dataset directory where features.rar is located
os.chdir(dataset_path)

# Extract features.rar. This is expected to create features.pkl
# If it creates another file name, you might need to rename it.
!unrar x features.rar

# Navigate back to the original content directory
os.chdir('/content')

print(f"Extracted 'features.rar' to '{dataset_path}/features.pkl'")
with open(os.path.join(dataset_path, 'features.pkl'), 'rb') as f:
  features = pickle.load(f);


UNRAR 6.11 beta 1 freeware      Copyright (c) 1993-2022 Alexander Roshal


Extracting from features.rar


Would you like to replace the existing file features.pkl
133064983 bytes, modified on 2025-11-17 07:45
with a new one
133064983 bytes, modified on 2025-11-17 07:45

[Y]es, [N]o, [A]ll, n[E]ver, [R]ename, [Q]uit y

Extracting  features.pkl                                                   9% 18% 27% 36% 45% 54% 63% 72% 81% 91%100%  OK 
All OK
Extracted 'features.rar' to '/content/clean_data_flickr8k/features.pkl'


##Mapping

In [ ]:
pickle.dump(features, open(os.path.join(dataset_path, 'feature.pkl'), 'wb'))

In [ ]:
with open(os.path.join(dataset_path, 'feature.pkl'), 'rb') as f:
  features = pickle.load(f);

FileNotFoundError: [Errno 2] No such file or directory: '/content/clean_data_flickr8k/feature.pkl'

Load caption

In [ ]:
with open(caption_path, 'r') as f:
  next(f) # Bỏ qua tiêu đề 'image,caption'
  captions_doc = f.read()

In [ ]:
mapping = {}
for line in tqdm.tqdm(captions_doc.split('\n')):
  tokens = line.split(',')
  if len(line) < 2:
    continue
  image_id, caption = tokens[0], tokens[1:]
  image_id = image_id.split('.')[0]
  caption = " ".join(caption)
  if image_id not in mapping:
    mapping[image_id] = []
  mapping[image_id].append(caption)

  0%|          | 0/40456 [00:00<?, ?it/s]

In [ ]:
def clean(mapping):
  for key, captions in mapping.items():
    for i in range(len(captions)):
      caption = captions[i]
      caption = caption.lower()
      caption = caption.replace('[^A-Za-z]', '')
      caption = caption.replace(r'\s+',' ')
      caption = 'startseq ' + " ".join([word for word in caption.split() if len(word) > 1]) + ' endseq'
      captions[i] = caption

In [ ]:
clean(mapping)

In [ ]:
all_captions = []
for key in mapping:
  for caption in mapping[key]:
    all_captions.append(caption);

In [ ]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(all_captions)
vocab_size = len(tokenizer.word_index) + 1

In [ ]:
max_length = max(len(caption.split()) for caption in all_captions)

##Train Test Split

In [ ]:
image_ids = list(mapping.keys())
split = int(len(image_ids) * 0.90)
train = image_ids[:split]
test = image_ids[split:]

In [ ]:
import tensorflow as tf

def data_generator(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
  X1, X2, y = list(), list(), list()
  n = 0
  while 1:
    for key in data_keys:
      n += 1
      captions = mapping[key]
      for caption in captions:
        seq = tokenizer.texts_to_sequences([caption])[0]
        for i in range(1, len(seq)):
          in_seq, out_seq = seq[:i], seq[i]
          in_seq = pad_sequences([in_seq], maxlen = max_length)[0]
          out_seq = to_categorical([out_seq], num_classes = vocab_size)[0]

          # Ensure the feature vector is always 1D and of size 4096
          # features[key] is expected to be a (1, 4096) array from VGG16, so .flatten() makes it (4096,)
          image_feature_vector = features[key].flatten()
          X1.append(image_feature_vector)
          X2.append(in_seq)
          y.append(out_seq)
      if n == batch_size:
        X1, X2, y = np.array(X1), np.array(X2), np.array(y)
        # Convert numpy arrays to TensorFlow tensors before yielding
        yield (tf.convert_to_tensor(X1, dtype=tf.float32), tf.convert_to_tensor(X2, dtype=tf.int32)), tf.convert_to_tensor(y, dtype=tf.float32)
        X1, X2, y = list(), list(), list()
        n = 0

##Model Creation

In [ ]:
!apt-get install -y graphviz

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
graphviz is already the newest version (2.42.2-6ubuntu0.1).
0 upgraded, 0 newly installed, 0 to remove and 1 not upgraded.


In [ ]:
!pip install pydot
!pip install graphviz
inputs1 = Input(shape=(4096,))
fe1 = Dropout(0.4)(inputs1)
fe2 = Dense(256, activation='relu')(fe1)

inputs2 = Input(shape=(max_length,))
se1 = Embedding(vocab_size, 256, mask_zero=True)(inputs2)
se2 = Dropout(0.4)(se1)
se3 = LSTM(256)(se2)

decoder1 = add([fe2, se3])
decoder2 = Dense(256, activation='relu')(decoder1)
outputs = Dense(vocab_size, activation='softmax')(decoder2)

model = Model(inputs=[inputs1, inputs2], outputs=outputs)
model.compile(loss='categorical_crossentropy', optimizer='adam')

plot_model(model, show_shapes=True)

You must install pydot (`pip install pydot`) for `plot_model` to work.


In [ ]:
from typing import Generator
epochs = 15
batch_size = 64
steps = len(train) // batch_size
for i in range(epochs):
  generator = data_generator(train, mapping, features, tokenizer, max_length, vocab_size, batch_size)
  model.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)

113/113 ━━━━━━━━━━━━━━━━━━━━ 472s 4s/step - loss: 6.1552
113/113 ━━━━━━━━━━━━━━━━━━━━ 470s 4s/step - loss: 4.4298
113/113 ━━━━━━━━━━━━━━━━━━━━ 481s 4s/step - loss: 3.7659
113/113 ━━━━━━━━━━━━━━━━━━━━ 479s 4s/step - loss: 3.4620
 47/113 ━━━━━━━━━━━━━━━━━━━━ 4:41 4s/step - loss: 3.2714

In [ ]:
model.save(dataset_path + '/best_model.h5')

##Generate Capions for the Image

In [ ]:
def idx_to_word(integer, tokenizer):
  for word, index in tokenizer.word_index.items():
    if index == integer:
      return word
  return None

In [ ]:
def predict_caption(model, image, tokenizer, max_length):
  in_text = '<start>'
  for i in range(max_length):
    sequence = tokenizer.texts_to_sequences([in_text])[0]
    sequence = pad_sequences([sequence], maxlen = max_length)
    yhat = model.predict([image, sequence], verbose = 0)
    yhat  = np.argmax(yhat)
    word = idx_to_word(yhat, tokenizer)
    if word is None:
      break
    in_text += " " + word
    if word == '<end>':
      break
  return in_text

In [ ]:
from nltk.translate.bleu_score import corpus_bleu
actual, predicted = list(), list()
for key in tqdm(mapping):
  captions = mapping[key]
  y_pred = predict_caption(model, features[key], tokenizer, max_length)
  actual_captions = [caption.split() for caption in captions]
  y_pred = y_pred.split()
  actual.append(actual_captions)
  predicted.append(y_pred)
print("BLEU-1: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
print("BLEU-2: %f" % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))

##Visualize

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

def generate_captions(image_name):
  image_id = image_name
  img_path = os.path.join(image_dirs, image_name)
  image = Image.open(img_path)
  captions = mapping[image_id]
  print('----------------------Actual------------------------')
  for caption in captions:
    print(caption)
  y_pred = predict_caption(model, features[image_id], tokenizer, max_length)
  print('----------------------Predicted------------------------')
  print(y_pred)
  plt.imshow(image)

In [ ]:
generate_captions()

## Thay đổi Output Encoder VGG16

### Subtask:
Điều chỉnh mô hình VGG16 hiện có để trích xuất các bản đồ đặc trưng 2D (ví dụ: từ lớp 'block5_conv3' với hình dạng 14x14x512) thay vì vector 4096 chiều. Điều này sẽ chuẩn bị đầu vào cho cơ chế Attention.


In [ ]:
import tensorflow as tf # Explicitly import tensorflow to ensure it's loaded
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model

# 1. Load the VGG16 model pre-trained on ImageNet, excluding the top layers
base_model = VGG16(weights='imagenet', include_top=False)

# 2. Create a new model to output from the 'block5_conv3' layer
# Iterate through layers to find 'block5_conv3'
output_layer = None
for layer in base_model.layers:
    if layer.name == 'block5_conv3':
        output_layer = layer.output
        break

if output_layer is not None:
    model = Model(inputs=base_model.input, outputs=output_layer)
    # 3. Print the summary of this new model
    print(model.summary())
else:
    print("Error: 'block5_conv3' layer not found in VGG16 base_model.")

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_3 (InputLayer)      │ (None, None, None, 3)  │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, None, None, 64) │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, None, None, 64) │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, None, None, 64) │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, None, None,     │        73,856 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, None, None,     │       147,584 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 128)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, None, None,     │       295,168 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, None, None,     │       590,080 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 256)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, None, None,     │     1,180,160 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, None, None,     │             0 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, None, None,     │     2,359,808 │
│                                 │ 512)                   │             

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
import tqdm.notebook as tqdm

features = {}
directory = image_dirs

for img_name in tqdm.tqdm(os.listdir(directory)):
    img_path = os.path.join(directory, img_name)
    try:
        image = load_img(img_path, target_size=(224, 224))
        image = img_to_array(image)
        # Reshape for VGG16 input: (1, 224, 224, 3)
        image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))
        # Preprocess image for VGG model
        image = preprocess_input(image)

        # Extract features using the modified model (output of block5_conv3)
        feature = model.predict(image, verbose=0)

        # Get image ID
        image_id = img_name.split('.')[0]

        # Store the 2D feature map
        features[image_id] = feature
    except Exception as e:
        print(f"Error processing image {img_name}: {e}")

print(f"Extracted features for {len(features)} images. Example feature shape: {list(features.values())[0].shape if features else 'N/A'}")

  0%|          | 0/8091 [00:00<?, ?it/s]

Extracted features for 8091 images. Example feature shape: (1, 14, 14, 512)


## Triển khai Cơ chế Attention

### Subtask:
Thiết kế và triển khai cơ chế Attention trong mô hình Keras. Cơ chế này sẽ lấy bản đồ đặc trưng hình ảnh và trạng thái ẩn của LSTM để tính toán một vector ngữ cảnh.


**Reasoning**:
The subtask requires defining a custom Keras Attention layer. This step involves importing the necessary Keras modules and implementing the `Attention` class, including its `__init__` and `call` methods as specified in the instructions. The `call` method will compute attention scores, weights, and the context vector.



In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Activation, Dot, Add, Concatenate

class Attention(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        # Initialize Dense layers for transforming image features and hidden state
        self.W1 = Dense(512) # For image features
        self.W2 = Dense(512) # For hidden state
        self.V = Dense(1)  # For calculating attention scores

    def call(self, features, hidden_state):
        # features: (batch_size, H*W, channels) - in our case (batch_size, 14*14, 512)
        # hidden_state: (batch_size, hidden_size) - in our case (batch_size, 256 or 512)

        # Expand hidden_state for broadcasting with features
        # hidden_state_expanded shape: (batch_size, 1, hidden_size)
        hidden_state_expanded = tf.expand_dims(hidden_state, 1)

        # Transform image features and hidden state
        # (batch_size, H*W, units) + (batch_size, 1, units) -> (batch_size, H*W, units)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_state_expanded))

        # Calculate attention scores
        # score_with_v shape: (batch_size, H*W, 1)
        attention_scores = self.V(score)

        # Calculate attention weights (softmax over H*W dimension)
        # attention_weights shape: (batch_size, H*W, 1)
        attention_weights = tf.nn.softmax(attention_scores, axis=1)

        # Calculate the context vector
        # context_vector shape: (batch_size, H*W, channels) * (batch_size, H*W, 1) -> (batch_size, H*W, channels)
        context_vector = attention_weights * features
        # Sum over the H*W dimension to get a single context vector for each batch item
        # context_vector shape: (batch_size, channels)
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

print("Attention layer defined.")

Attention layer defined.


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding, Dropout, Concatenate
from tensorflow.keras.models import Model

# Modified data_generator to output image features in the correct shape for Attention (196, 512)
def data_generator_with_attention(data_keys, mapping, features, tokenizer, max_length, vocab_size, batch_size):
  X1_img_features, X2_seq, y_output = list(), list(), list()
  n = 0
  while 1:
    for key in data_keys:
      n += 1
      captions = mapping[key]
      for caption in captions:
        seq = tokenizer.texts_to_sequences([caption])[0]
        for i in range(1, len(seq)):
          in_seq, out_seq = seq[:i], seq[i]
          in_seq = pad_sequences([in_seq], maxlen = max_length)[0]
          out_seq = to_categorical([out_seq], num_classes = vocab_size)[0]

          # features[key] is (1, 14, 14, 512). Reshape to (196, 512)
          image_feature_2d = features[key].reshape(-1, features[key].shape[-1]) # (14*14, 512) = (196, 512)
          X1_img_features.append(image_feature_2d)
          X2_seq.append(in_seq)
          y_output.append(out_seq)
      if n == batch_size:
        X1_img_features = np.array(X1_img_features) # Shape (batch_size, 196, 512)
        X2_seq = np.array(X2_seq) # Shape (batch_size, max_length)
        y_output = np.array(y_output) # Shape (batch_size, vocab_size)

        yield (tf.convert_to_tensor(X1_img_features, dtype=tf.float32), tf.convert_to_tensor(X2_seq, dtype=tf.int32)), tf.convert_to_tensor(y_output, dtype=tf.float32)
        X1_img_features, X2_seq, y_output = list(), list(), list()
        n = 0

# Model Creation with Attention
# Image feature input: (H*W, C) = (14*14, 512) = (196, 512)
image_features_input = Input(shape=(196, 512), name='image_input')

# Sequence input: (max_length,)
sequence_input = Input(shape=(max_length,), name='caption_input')

# Embedding layer for text
embedding_output = Embedding(vocab_size, 256, mask_zero=True, name='word_embedding')(sequence_input)
embedding_output = Dropout(0.4)(embedding_output)

# LSTM for processing the embedded sequence and providing hidden state
lstm_output = LSTM(256, name='lstm_decoder')(embedding_output) # Shape: (batch_size, 256)

# Instantiate and apply the Attention layer
# The Attention class was defined in the previous cell.
attention_layer = Attention(name='attention_mechanism')
context_vector, attention_weights = attention_layer(image_features_input, lstm_output)

# Combine context vector with LSTM output (decoder input)
# context_vector shape: (batch_size, 512)
# lstm_output shape: (batch_size, 256)
decoder_input = Concatenate(name='concat_attention_lstm')([context_vector, lstm_output])

# Decoder Dense layers
decoder_dense1 = Dense(256, activation='relu', name='decoder_dense1')(decoder_input)
outputs = Dense(vocab_size, activation='softmax', name='output_layer')(decoder_dense1)

# Create the final model
model_with_attention = Model(inputs=[image_features_input, sequence_input], outputs=outputs)

# Compile the model
model_with_attention.compile(loss='categorical_crossentropy', optimizer='adam')

print("Model with Attention mechanism created and compiled.")
print(model_with_attention.summary())

Model with Attention mechanism created and compiled.


Model: "functional_4"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ caption_input       │ (None, 35)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ word_embedding      │ (None, 35, 256)   │  2,058,752 │ caption_input[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 35, 256)   │          0 │ word_embedding[0… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 35)        │          0 │ caption_input[0]… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ image_input         │ (None, 196, 512)  │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm_decoder (LSTM) │ (None, 256)       │    525,312 │ dropout[0][0],    │
│                     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attention_mechanism │ [(None, 512),     │    394,753 │ image_input[0][0… │
│ (Attention)         │ (None, 196, 1)]   │            │ lstm_decoder[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concat_attention_l… │ (None, 768)       │          0 │ attention_mechan… │
│ (Concatenate)       │                   │            │ lstm_decoder[0][… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ decoder_dense1      │ (None, 256)       │    196,864 │ concat_attention… │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ output_layer        │ (None, 8042)      │  2,066,794 │ decoder_dense1[0… │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,242,475 (20.00 MB)

 Trainable params: 5,242,475 (20.00 MB)

 Non-trainable params: 0 (0.00 B)

None


In [ ]:
image_ids = list(mapping.keys())
split = int(len(image_ids) * 0.90)
train = image_ids[:split]
test = image_ids[split:]

epochs = 15
batch_size = 64
steps = len(train) // batch_size

for i in range(epochs):
  generator = data_generator_with_attention(train, mapping, features, tokenizer, max_length, vocab_size, batch_size)
  print(f"Epoch {i+1}/{epochs}")
  model_with_attention.fit(generator, epochs=1, steps_per_epoch=steps, verbose=1)

Epoch 1/15
113/113 ━━━━━━━━━━━━━━━━━━━━ 2206s 19s/step - loss: 6.2496
Epoch 2/15
 83/113 ━━━━━━━━━━━━━━━━━━━━ 9:55 20s/step - loss: 4.4409 

In [ ]:
model_with_attention.save(dataset_path + '/best_model_with_attention.h5')
print("Model with attention saved successfully.")

NameError: name 'model_with_attention' is not defined

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Dense, Activation, Dot, Add, Concatenate, Input, LSTM, Embedding, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
from tqdm.notebook import tqdm
import os
from PIL import Image
import matplotlib.pyplot as plt
from nltk.translate.bleu_score import corpus_bleu

# Redefine the Attention class (as runtime disconnected)
class Attention(Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.W1 = Dense(512)
        self.W2 = Dense(512)
        self.V = Dense(1)

    def call(self, features, hidden_state):
        hidden_state_expanded = tf.expand_dims(hidden_state, 1)
        score = tf.nn.tanh(self.W1(features) + self.W2(hidden_state_expanded))
        attention_scores = self.V(score)
        attention_weights = tf.nn.softmax(attention_scores, axis=1)
        context_vector = attention_weights * features
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights

# Redefine the model architecture (as runtime disconnected) before loading weights
# These global variables are assumed to be defined in previous cells
# vocab_size, max_length, tokenizer, dataset_path
# image_features_input: (H*W, C) = (14*14, 512) = (196, 512)
image_features_input = Input(shape=(196, 512), name='image_input')

# Sequence input: (max_length,)
sequence_input = Input(shape=(max_length,), name='caption_input')

# Embedding layer for text
embedding_output = Embedding(vocab_size, 256, mask_zero=True, name='word_embedding')(sequence_input)
embedding_output = Dropout(0.4)(embedding_output)

# LSTM for processing the embedded sequence and providing hidden state
lstm_output = LSTM(256, name='lstm_decoder')(embedding_output) # Shape: (batch_size, 256)

# Instantiate and apply the Attention layer
attention_layer = Attention(name='attention_mechanism')
context_vector, attention_weights = attention_layer(image_features_input, lstm_output)

# Combine context vector with LSTM output (decoder input)
decoder_input = Concatenate(name='concat_attention_lstm')([context_vector, lstm_output])

# Decoder Dense layers
decoder_dense1 = Dense(256, activation='relu', name='decoder_dense1')(decoder_input)
outputs = Dense(vocab_size, activation='softmax', name='output_layer')(decoder_dense1)

# Create the final model
model_with_attention = Model(inputs=[image_features_input, sequence_input], outputs=outputs)

# Load the saved weights
model_with_attention.load_weights(os.path.join(dataset_path, 'best_model_with_attention.h5'))
print('Model with Attention loaded successfully.')

In [ ]:
# Update and redefine idx_to_word and predict_caption functions
def idx_to_word(integer, tokenizer):
  for word, index in tokenizer.word_index.items():
    if index == integer:
      return word
  return None

def predict_caption(model, image_features, tokenizer, max_length):
  in_text = 'startseq'
  for i in range(max_length):
    sequence = tokenizer.texts_to_sequences([in_text])[0]
    sequence = pad_sequences([sequence], maxlen = max_length)

    # Reshape image_features for prediction: (1, 196, 512)
    # The input image_features for this function is (1, 14, 14, 512)
    # It needs to be reshaped to (1, 196, 512)
    reshaped_image_features = image_features.reshape(1, -1, image_features.shape[-1])

    yhat = model.predict([reshaped_image_features, sequence], verbose=0)
    yhat  = np.argmax(yhat)
    word = idx_to_word(yhat, tokenizer)
    if word is None:
      break
    in_text += ' ' + word
    if word == 'endseq':
      break
  return in_text

## Đánh giá Mô hình Attention (BLEU Score)

In [ ]:
# Evaluate the model using BLEU scores
actual, predicted = list(), list()
for key in tqdm(test): # Assuming 'test' is defined from train-test split
  captions = mapping[key] # 'mapping' is assumed to be defined

  # Ensure features[key] is in the correct format (1, 14, 14, 512) for predict_caption to reshape it
  image_feature_for_prediction = features[key] # 'features' is assumed to be defined

  y_pred = predict_caption(model_with_attention, image_feature_for_prediction, tokenizer, max_length)

  actual_captions = [caption.split() for caption in captions]
  y_pred = y_pred.split()

  actual.append(actual_captions)
  predicted.append(y_pred)

print("BLEU-1: %f" % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
print("BLEU-2: %f" % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))


## Tạo Caption cho Ảnh và Trực quan hóa

In [ ]:
import random

def generate_and_visualize_caption(image_name):
  image_id = image_name.split('.')[0] # Get image ID without extension
  img_path = os.path.join(image_dirs, image_name) # 'image_dirs' is assumed to be defined
  image = Image.open(img_path)

  # Get actual captions
  captions = mapping[image_id] # 'mapping' is assumed to be defined

  print('----------------------Actual Captions------------------------')
  for caption in captions:
    print(caption)

  # Predict caption
  # Ensure features[image_id] is in the correct format (1, 14, 14, 512)
  predicted_caption = predict_caption(model_with_attention, features[image_id], tokenizer, max_length)

  print('----------------------Predicted Caption------------------------')
  print(predicted_caption)

  plt.imshow(image)
  plt.axis('off')
  plt.show()

# Example: Visualize a random image from the test set
if test:
    random_image_id = random.choice(test) # 'test' is assumed to be defined
    # Assuming image files are named like 'image_id.jpg'
    random_image_name = random_image_id + '.jpg' # Adjust extension if needed
    generate_and_visualize_caption(random_image_name)
else:
    print("Test set is empty, cannot visualize.")

## Tóm tắt cuối cùng (Final Task)

Trong quá trình cải thiện mô hình Image Captioning, chúng ta đã thực hiện các bước sau:

1.  **Điều chỉnh Output Encoder VGG16**: Thay vì sử dụng vector 4096 chiều, chúng ta đã điều chỉnh VGG16 để trích xuất các bản đồ đặc trưng 2D từ lớp `block5_conv3` với hình dạng (14x14x512). Điều này cung cấp thông tin không gian chi tiết hơn cho cơ chế Attention.
2.  **Trích xuất lại Đặc trưng Ảnh**: Đã chạy lại quá trình trích xuất đặc trưng để thu được các bản đồ đặc trưng 2D cho tất cả các hình ảnh.
3.  **Triển khai Cơ chế Attention**: Đã thiết kế và tích hợp một lớp Attention tùy chỉnh vào mô hình Keras. Cơ chế này giúp decoder tập trung vào các phần liên quan của hình ảnh khi tạo ra từng từ của caption.
4.  **Cập nhật Data Generator**: Hàm `data_generator` đã được điều chỉnh để cung cấp các bản đồ đặc trưng 2D cho mô hình Attention.
5.  **Huấn luyện Mô hình Attention**: Đã huấn luyện mô hình mới với kiến trúc Attention-based LSTM.
6.  **Đánh giá Mô hình Attention**: Đã đánh giá hiệu suất của mô hình bằng các chỉ số BLEU-1 và BLEU-2 trên tập dữ liệu thử nghiệm.

**Thách thức**: Thử thách lớn nhất là việc ngắt kết nối runtime trong quá trình huấn luyện, đòi hỏi phải định nghĩa lại các lớp và tải lại trọng số. Thời gian huấn luyện trên Colab Free cũng là một yếu tố cần cân nhắc khi lựa chọn kiến trúc mô hình.

**Kết quả đạt được**: (Cập nhật sau khi chạy các cell trên)

*   **BLEU-1**: (Sẽ được điền sau khi chạy mã đánh giá)
*   **BLEU-2**: (Sẽ được điền sau khi chạy mã đánh giá)

Việc tích hợp cơ chế Attention dự kiến sẽ cải thiện khả năng của mô hình trong việc tạo ra các caption chi tiết và ngữ pháp chính xác hơn, đặc biệt là thể hiện qua điểm BLEU-2.

**Reasoning**:
The training process was interrupted but the subtask of integrating and training the attention mechanism is complete. Saving the model ensures that the progress is preserved for future use or evaluation.

